In [4]:
from sklearn.linear_model import LinearRegression,Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network._multilayer_perceptron import MLPRegressor
from statsmodels.tsa.statespace.sarimax import SARIMAX
from google.oauth2 import service_account
from datetime import datetime as date
from scipy.stats import pearsonr
from scipy.stats import spearmanr
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import TimeSeriesSplit 
from sklearn.metrics import r2_score,make_scorer,mean_squared_error
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import mutual_info_regression
from statsmodels.tsa.stattools import adfuller

import joblib
import pickle
import pandas as pd
import pandas_gbq
import numpy as np
import xgboost
import lightgbm
import os

### Gathering Data and Feature Engineering

In [5]:


#using shifted windows for rolling data to prevent data leakage
player_query = f""" 
SELECT *
from `capstone_data.player_modeling_data_partitioned`
order by game_date asc
"""

team_query = f"""
SELECT *
from `capstone_data.team_modeling_data_partitioned`
order by game_date asc
"""


In [6]:
 
try:
    full_data = pd.read_csv('full_data.csv')

except:
    nba_player_data = pd.DataFrame(pandas_gbq.read_gbq(player_query,project_id='miscellaneous-projects-444203',progress_bar_type='tqdm'))
    team_data = pd.DataFrame(pandas_gbq.read_gbq(team_query,project_id='miscellaneous-projects-444203',progress_bar_type='tqdm'))
    team_data  = team_data.merge(team_data,on='game_id',suffixes=('',"_opponent"))
    team_data = team_data[team_data["team_id"] != team_data["team_id_opponent"]]
    full_data = nba_player_data.merge(team_data, on = ['game_id','team'], how = 'inner',suffixes=('','remove'))
    full_data.drop([column for column in full_data.columns if 'remove' in column],axis = 1 , inplace=True) 
    full_data.drop([column for column in full_data.columns if '_1' in column],axis = 1 , inplace=True)
    full_data.to_csv('full_data.csv',mode = 'x')

Downloading: 100%|██████████|
Downloading: 100%|██████████|


In [7]:
data_ordered = full_data.sort_values('game_date')

data_ordered.dropna(inplace=True)


#### Feature Engineering Ideas 

* (ratio of 3pa and fga and 3pm and 3pa) TS% for players efg% 
* for players assist_to_turnover ratio assist ratio, 
* rebound_cahnce, defesnive reb %, 
* ast_ratio_season * pace, 
* home * pts season - data pts 3pm avg,
* cold_streak pts_3gm_avg < pts_season boolean, 
* away difficulty away * opponent_defrtg_3gm_avg,
* home_performance = data_ordered[data_ordered["home"] == 1].groupby("team")["pts_season"].mean()
* away_performance = data_ordered[data_ordered["away"] == 1].groupby("team")["pts_season"].mean() these would be to see how the team performance changes 


In [8]:
# data_ordered['pts_per_min_3gm'] = data_ordered['pts_3gm_avg']/data_ordered['min_3gm_avg']
# data_ordered['pts_per_min_season'] = data_ordered['pts_season']/data_ordered['min_season']
# data_ordered['pts_per_min_momentum'] = data_ordered['pts_per_min_3gm'] - data_ordered['pts_per_min_season']

# data_ordered['fg3m_per_min_3gm'] = data_ordered['fg3m_3gm_avg']/data_ordered['min_3gm_avg']
# data_ordered['fg3m_per_min_season'] = data_ordered['fg3m_season']/data_ordered['min_season']
# data_ordered['fg3m_per_min_momentum'] = data_ordered['fg3m_per_min_3gm'] - data_ordered['fg3m_per_min_season'] 

# data_ordered['reb_per_min_3gm'] = data_ordered['reb_3gm_avg']/data_ordered['min_3gm_avg']
# data_ordered['reb_per_min_season'] = data_ordered['reb_season']/data_ordered['min_season']
# data_ordered['reb_per_min_momentum'] = data_ordered['fg3m_per_min_3gm'] - data_ordered['reb_per_min_season']


In [9]:
data_ordered = data_ordered.groupby(['player','season']).apply(lambda x: x.iloc[3:]).reset_index(drop=True)

/var/folders/0r/xm2d9wcx2w93pm220m9lfw3r0000gn/T/ipykernel_3243/3943372088.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  data_ordered = data_ordered.groupby(['player','season']).apply(lambda x: x.iloc[3:]).reset_index(drop=True)


In [10]:
data_ordered.sort_values(by='game_date',inplace=True)

In [11]:
data_ordered['game_date'] = pd.to_datetime(data_ordered['game_date'])

In [12]:
data_ordered['days_ago'] = (data_ordered['game_date'].max() - data_ordered['game_date']).dt.days
data_ordered['time_decay_weight'] = 1 / (1 + np.log(1 + data_ordered['days_ago']))

In [13]:
pd.set_option('display.max_columns',100000)

In [14]:
try:
    data_ordered = data_ordered.drop('Unnamed: 0', axis =1)
except KeyError:
    print('Irregular column not made')

Irregular column not made


In [15]:
# Fill NaNs with the column mean, but only for numeric columns
data_ordered.fillna(data_ordered.select_dtypes(include=['number']).mean(), inplace=True)


In [16]:
numeric_columns = data_ordered.select_dtypes(include=['number']).columns.tolist()
numeric_columns = [column for column in numeric_columns if column not in ['pts','reb','ast','blk','stl','3pm','game_id','game_date','days_ago','time_decay_weight','team_id', "gp_rank", "w_rank", "l_rank", "w_pct_rank", "min_rank", "fgm_rank",
    "fga_rank", "fg_pct_rank", "fg3m_rank", "fg3a_rank", "fg3_pct_rank",
    "ftm_rank", "fta_rank", "ft_pct_rank", "oreb_rank", "dreb_rank",
    "reb_rank", "ast_rank", "tov_rank", "stl_rank", "blk_rank",
    "blka_rank", "pf_rank", "pfd_rank", "pts_rank", "plus_minus_rank",]]

numeric_columns = [feature for feature in numeric_columns if any(keyword in feature for keyword in ["3gm_avg", "season", "momentum"])]
features = {feature:[] for feature in ['pts','reb','ast','3pm']}

In [17]:
split_index = int(len(data_ordered) * .80)

train_data = data_ordered.iloc[:split_index]
test_data = data_ordered[split_index:]

In [18]:


for category in features.keys():
    x = train_data[numeric_columns]
    y = train_data[category]

    mi_scores = mutual_info_regression(x, y)
    mi_scores = pd.Series(mi_scores, index=numeric_columns)
    selected_features = mi_scores[mi_scores > 0.10].index.tolist()  

    features[category] = selected_features


In [19]:
#These values appeared to have non-linear relationships applying transformations
# data_ordered['ft%_season'] = np.log1p(data_ordered['ft%_season'])
# data_ordered['stl_3gm_avg'] = np.log1p(data_ordered['stl_3gm_avg'])
# data_ordered['stl_season'] = np.log1p(data_ordered['stl_season'])
# data_ordered['to_season'] = data_ordered['to_season']**2 
# data_ordered['to_3gm_avg'] = data_ordered['to_3gm_avg']**2 

In [20]:
tscv = TimeSeriesSplit(n_splits=5)

In [21]:
saved_models = {category:{} for category in ['pts','reb','ast','3pm']} 
saved_results = {category:{} for category in ['pts','reb','ast','3pm']}

#### SHAP
Applying shap to help reduce collinearity

### Linear Model

In [22]:

for category in features.keys():

    features_list = [f for f in features[category] if f != category]
    print(len(features_list))
    x_train,y_train = train_data[features_list],train_data[category]
    x_test, y_test = test_data[features_list],test_data[category]
    linear_model = LinearRegression()

    linear_model.fit(x_train,y_train)

    y_pred = linear_model.predict(x_test)
    print(category)
    print(r2_score(y_true=y_test,y_pred=y_pred))

    saved_results[category]['linear_model']={'r2':{r2_score(y_true=y_test,y_pred=y_pred)}, 'mse':{mean_squared_error(y_true=y_test,y_pred=y_pred)}}
    saved_models[category]['linear_model'] = linear_model

34
pts
0.5934293780764077
16
reb
0.5260932708144328
16
ast
0.5673155402017072
9
3pm
0.39733046438590114


In [23]:
for category in features.keys():
    features_list = [f for f in features[category] if f != category]
    x_train,y_train = train_data[features_list],train_data[category]
    x_test, y_test = test_data[features_list],test_data[category]
    ridge_model = Ridge(alpha=1)

    ridge_model.fit(x_train,y_train)

    output = pd.DataFrame({'prediction':ridge_model.predict(x_test), 'actual':y_test})
    print(category)
    print(r2_score(y_true=output['actual'],y_pred=output['prediction']))

pts
0.5934267704328423
reb
0.5261121726866936
ast
0.5673155430468425
3pm
0.3973304754355177


In [24]:
# from sklearn.feature_selection import RFE
# from sklearn.linear_model import Ridge, Lasso

# features = {}

# for category in ['pts', 'reb', 'ast', '3pm']:
#     x_train = train_data[numeric_columns]
#     y_train = train_data[category]

#     # Use Lasso to select features automatically
#     lasso = Lasso(alpha=0.01)  # Adjust alpha based on tuning
#     lasso.fit(x_train, y_train)

#     # Keep only features with nonzero coefficients
#     selected_features = [f for f, coef in zip(numeric_columns, lasso.coef_) if coef != 0]

#     # Cap the number of features (e.g., max 50)
#     selected_features = selected_features[:min(len(selected_features), 50)]

#     features[category] = selected_features

In [25]:
from sklearn.model_selection import cross_val_score

for category in features.keys():
    features_list = features[category]

    x_train, y_train = train_data[features_list], train_data[category]
    x_test, y_test = test_data[features_list], test_data[category]

    linear_model = Ridge(alpha=1.0)  # Use Ridge instead of LinearRegression
    linear_model.fit(x_train, y_train)

    # Cross-validation score instead of just test R²
    cv_r2 = cross_val_score(linear_model, x_train, y_train, cv=5, scoring='r2').mean()

    y_pred = linear_model.predict(x_test)
    test_r2 = r2_score(y_test, y_pred)

    print(f"{category}: Cross-Val R² = {cv_r2:.4f}, Test R² = {test_r2:.4f}")


pts: Cross-Val R² = 0.5718, Test R² = 0.5934
reb: Cross-Val R² = 0.5292, Test R² = 0.5261
ast: Cross-Val R² = 0.5805, Test R² = 0.5673
3pm: Cross-Val R² = 0.4002, Test R² = 0.3973


### XGboost

In [26]:
scaler = StandardScaler()

scaled_data = scaler.fit_transform(data_ordered[numeric_columns])

scaled_data_df = pd.DataFrame(scaled_data,columns=numeric_columns)

split_index = int(len(data_ordered) * .80)

scaled_train_data = scaled_data_df.iloc[:split_index]
scaled_test_data = scaled_data_df[split_index:]

In [27]:
param_grid = {'max_depth':[3,6,9],'learning_rate':[.01,.05,.1,.3],'booster':['gbtree','dart'],'subsample':[.5,.7,.9],'colsample_bytree':[.5,.7,.9],'n_estimators':[100,300,500]}
param_linear = {'booster':['gblinear'],'lambda':[0,.1,1,10],'alpha':[0,.1,1,10]}

In [28]:
xgb_regressor = xgboost.XGBRegressor()
mse_score = make_scorer(mean_squared_error,greater_is_better=False)
r2_scorer = make_scorer(r2_score)
scoring = {'MSE':mse_score,'r2':r2_scorer}
grid_search = GridSearchCV(estimator=xgb_regressor,param_grid=param_grid,scoring = scoring,cv=tscv,n_jobs=1,verbose=0,refit='r2')
grid_linear_search = GridSearchCV(estimator=xgb_regressor,param_grid=param_linear,scoring = scoring,cv=tscv,n_jobs=3,verbose=0,refit='r2')


In [29]:
xg_features =  features

In [30]:
for category in features.keys():
    x_train,y_train = scaled_train_data[xg_features[category]],train_data[category]
    x_test, y_test = scaled_test_data[xg_features[category]],test_data[category]

    fit_params = {'eval_set':[(x_test,y_test)],'early_stopping_rounds':20,'verbose':False}

    grid_linear_search.estimator.set_params(eval_metric='rmse')


    grid_linear_search.fit(x_train,y_train)


    print(category)
    print(grid_linear_search.best_params_)
    print(grid_linear_search.best_score_)

    y_pred = grid_linear_search.best_estimator_.predict(x_test)

    saved_models[category]['XGboost'] = grid_linear_search.best_estimator_
    saved_results[category]['XGboost']={'r2':{r2_score(y_true=y_test,y_pred=y_pred)}, 'mse':{mean_squared_error(y_true=y_test,y_pred=y_pred)}}


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


pts
{'alpha': 0, 'booster': 'gblinear', 'lambda': 0}
0.5737464460209214


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


reb
{'alpha': 0, 'booster': 'gblinear', 'lambda': 0}
0.5302166165189289


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


ast
{'alpha': 0, 'booster': 'gblinear', 'lambda': 0}
0.5787785382639328


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


3pm
{'alpha': 0, 'booster': 'gblinear', 'lambda': 0}
0.3985515807574412


### LightGBM

In [31]:
light = lightgbm.LGBMRegressor(boosting_type='gbdt', n_estimators=500)  # Using hist for faster training

param_grid = {
    'num_leaves': [15, 31, 50, 75],           # control model complexity
    'learning_rate': [0.005, 0.01, 0.05],     # finer learning rate control
    'max_depth': [-1, 5, 10, 15],             # more depth range
    'min_child_samples': [10, 20, 30],        # regularization (data per leaf)
    'subsample': [0.8, 1.0],                  # for row sampling
    'colsample_bytree': [0.8, 1.0],           # for feature sampling
}


In [32]:
from sklearn.model_selection import RandomizedSearchCV, TimeSeriesSplit
from lightgbm import LGBMRegressor
import numpy as np

# Define the model
lgb_model = LGBMRegressor(n_estimators=1000, random_state=42,verbosity=-1)

# Define the expanded parameter grid
param_grid = {
    'num_leaves': [15, 31, 50, 75],
    'learning_rate': [0.005, 0.01, 0.05],
    'max_depth': [-1, 5, 10, 15],
    'min_child_samples': [10, 20, 30],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
}

# Time series split (if your data is chronological)
tscv = TimeSeriesSplit(n_splits=5)

# Randomized search setup
random_search = RandomizedSearchCV(
    estimator=lgb_model,
    param_distributions=param_grid,
    n_iter=40,  # control number of total combinations to test
    cv=tscv,
    scoring='r2',
    verbose=0,
    n_jobs=-1,
    random_state=42
)

# Fit to your training data
# Best model + params


In [33]:
light_grid_search = GridSearchCV(estimator=light,param_grid=param_grid,cv=tscv,verbose=-1,n_jobs=4)

In [34]:
split_index = int(len(data_ordered) * .80)

train_data = data_ordered.iloc[:split_index]
test_data = data_ordered[split_index:]
for category in features.keys():
    x_train,y_train = train_data[features[category]],train_data[category]
    x_test,y_test = test_data[features[category]],test_data[category]

    random_search.fit(x_train,y_train)

    best_model = random_search.best_estimator_
    print(category)
    print("Best Parameters:", random_search.best_params_)

    y_pred = best_model.predict(x_test)

    mse = mean_squared_error(y_test,y_pred)
    r2 = r2_score(y_test,y_pred)

    saved_models[category]['lightgbm'] = best_model
    print(f'MSE: {mse}')
    print(f'R2: {r2}')

    saved_results[category]['lightgbm']={'r2':{r2_score(y_true=y_test,y_pred=y_pred)}, 'mse':{mean_squared_error(y_true=y_test,y_pred=y_pred)}}


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


pts
Best Parameters: {'subsample': 0.8, 'num_leaves': 75, 'min_child_samples': 30, 'max_depth': -1, 'learning_rate': 0.01, 'colsample_bytree': 0.8}
MSE: 30.47468163396413
R2: 0.6154665727797345


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


reb
Best Parameters: {'subsample': 0.8, 'num_leaves': 15, 'min_child_samples': 20, 'max_depth': 15, 'learning_rate': 0.01, 'colsample_bytree': 0.8}
MSE: 5.650851638903141
R2: 0.5306307327233127
ast
Best Parameters: {'subsample': 1.0, 'num_leaves': 15, 'min_child_samples': 30, 'max_depth': 5, 'learning_rate': 0.01, 'colsample_bytree': 1.0}
MSE: 3.029610278592103
R2: 0.5695801765858595
3pm
Best Parameters: {'subsample': 0.8, 'num_leaves': 75, 'min_child_samples': 20, 'max_depth': 15, 'learning_rate': 0.01, 'colsample_bytree': 1.0}
MSE: 1.2675096918044908
R2: 0.44695450531651626


In [35]:
#Ensemble modeling

models = joblib.load('models.pkl')

linear_models = {cat: models[cat]['linear_model'] for cat in models if 'linear_model' in models[cat]}
lightgbm_models = {cat: models[cat]['lightgbm'] for cat in models if 'lightgbm' in models[cat]}

In [39]:
joblib.dump(saved_models,'models.pkl')


['models.pkl']

In [40]:
with open('saved_performance.txt', 'w') as file:
    for category, models in saved_results.items():
        file.write(f"Category: {category}\n")
        for model, metrics in models.items():
            file.write(f"  Model: {model}\n")
            for metric, value in metrics.items():
                file.write(f"    {metric}: {value}\n")
        file.write("\n")  # Newline between categories
